<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    h1 {
      text-align: center;
    }
  </style>
</head>
<body>

<h1>Transfer Learning with MobileNetV2 🤖📸</h1>

</body>
</html>

<img src="https://dataaspirant.com/wp-content/uploads/2023/03/3-8.png" width="750">

# 1. Introduction ℹ️
### Transfer learning is a machine learning technique where a model trained on one task is re-purposed on a second related task. 
### In this notebook, we'll explore how to use transfer learning with the MobileNetV2 architecture for brain MRI tumour detection.

## *1.1 What is Transfer Learning? 🔄*
#### Transfer learning involves leveraging the knowledge gained while solving one problem and applying it to a different, but related problem.
#### Instead of starting the learning process from scratch, transfer learning allows us to use pre-trained models as a starting point and then fine-tune them for our specific task.

## *1.2 Advantages of Transfer Learning:*
#### **Reduced Training Time**: Transfer learning significantly reduces training time since we start with pre-trained weights that have already learned meaningful features.
#### **Less Data Required**: It allows us to achieve good performance even with less labeled data, as compared to training a model from scratch.
#### **Improved Generalization**: Pre-trained models have learned features from a diverse range of data, which often leads to better generalization on new tasks.

## *1.3 Use Cases of Transfer Learning:*
### Transfer learning has found applications in various domains, including:

#### 1. **Image Classification**: Classifying images into different categories.
#### 2. **Object Detection**: Identifying and locating objects within images.
#### 3. **Natural Language Processing (NLP)**: Tasks such as sentiment analysis, text classification, and language translation.
#### 4. **Healthcare**: Diagnosing diseases from medical images like MRI scans and X-rays.

## 2. Importing tools (I mean libraries and packages ) ⚙️📦

##### Let's start by importing the necessary libraries and packages for our project.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import shutil
import random

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16,EfficientNetV2M

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

2024-04-13 10:44:38.104745: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 10:44:38.104833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 10:44:38.224789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 3. Restructuring the training and validation directories

In [2]:
# Define paths to your "yes" and "no" folders and training/validation folders
yes_folder = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes'
no_folder = '/kaggle/input/brain-mri-images-for-brain-tumor-detection/no'
training_folder = 'training_folder'
validation_folder = 'validation_folder'

# Create training and validation folders if they don't exist
os.makedirs(os.path.join(training_folder, 'yes'), exist_ok=True)
os.makedirs(os.path.join(training_folder, 'no'), exist_ok=True)
os.makedirs(os.path.join(validation_folder, 'yes'), exist_ok=True)
os.makedirs(os.path.join(validation_folder, 'no'), exist_ok=True)

# Function to split images and copy them to training and validation folders
def split_and_copy(source_folder, training_folder, validation_folder, split_size):
    # List all images in the source folder
    images = [os.path.join(source_folder, img) for img in os.listdir(source_folder) if img.endswith('.jpg') or img.endswith('.png')]
    
    # Calculate the number of images to be copied to training and validation sets
    num_images = len(images)
    num_training_images = int(split_size * num_images)
    num_validation_images = num_images - num_training_images
    
    # Randomly shuffle the list of images
    random.shuffle(images)
    
    # Copy images to training folder
    for img_path in images[:num_training_images]:
        img_name = os.path.basename(img_path)
        destination_path = os.path.join(training_folder, img_name)
        shutil.copy(img_path, destination_path)
        
    # Copy images to validation folder
    for img_path in images[num_training_images:]:
        img_name = os.path.basename(img_path)
        destination_path = os.path.join(validation_folder, img_name)
        shutil.copy(img_path, destination_path)

# Split and copy images from "yes" folder
split_and_copy(yes_folder, os.path.join(training_folder, 'yes'), os.path.join(validation_folder, 'yes'), 0.8)

# Split and copy images from "no" folder
split_and_copy(no_folder, os.path.join(training_folder, 'no'), os.path.join(validation_folder, 'no'), 0.8)

print("Images copied successfully.")

Images copied successfully.


# 4. Data loading and deta pre-processing

In [3]:
# Define paths to your dataset
train_data_dir = '/kaggle/working/training_folder'
validation_data_dir = '/kaggle/working/validation_folder'

# Define image dimensions
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)

# Number of classes
num_classes = 2

# Batch size
batch_size = 32

# Preprocess data and augment images
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 137 images belonging to 2 classes.
Found 36 images belonging to 2 classes.


# 5. Model Architecture

## *5.1 Loading the pre-trained MobileNetV2 model*

In [4]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## *5.2 Add custom classification layers*

In [5]:
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

# Create final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Model training

In [6]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Save the model
model.save('brain_mri_transfer_learning_model.h5')

Epoch 1/10
1/4 ━━━━━━━━━━━━━━━━━━━━ 41s 14s/step - accuracy: 0.3333 - loss: 2.3949

I0000 00:00:1713005105.344656      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713005105.378579      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4173 - loss: 19.1271

W0000 00:00:1713005113.623198      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.4233 - loss: 22.2322 - val_accuracy: 0.8750 - val_loss: 4.1209
Epoch 2/10
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7812 - loss: 6.8331

W0000 00:00:1713005117.054504      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.7812 - loss: 6.8331 - val_accuracy: 0.7500 - val_loss: 22.2976
Epoch 3/10


W0000 00:00:1713005121.101587      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.6732 - loss: 15.3138 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9062 - loss: 5.3652 - val_accuracy: 0.7812 - val_loss: 6.4895
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.8235 - loss: 6.4816 - val_accuracy: 0.7500 - val_loss: 29.3987
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8438 - loss: 4.8688 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8248 - loss: 4.0843 - val_accuracy: 0.8125 - val_loss: 4.3363
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9375 - loss: 2.8820 - val_accuracy: 1.0000 - val_loss: 0.0046
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8750 - loss: 4.4862 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8750 - loss: 1.9729 - val_accuracy: 0.7812 - val

# 7.Conclusion 🎉

#### In this notebook, we explored the concept of transfer learning and its application using the MobileNetV2 architecture. We learned that transfer learning can significantly reduce training time, require less labeled data, and improve model generalization.
#### By leveraging pre-trained models like MobileNetV2, we can efficiently tackle image classification tasks with high accuracy and efficiency.

#### Transfer learning has become an indispensable tool in the machine learning toolkit, enabling practitioners to achieve remarkable results across various domains with less effort and computational resources.
#### As we continue to explore and experiment with transfer learning techniques, we can expect even more exciting advancements and applications in the field of deep learning.

### I hope this notebook has provided valuable insights and inspiration for your future projects.
### Happy learning and experimenting with transfer learning! 🚀🔬

## Upvote, Fork, and Make Changes 👍🍴✏️
### If you found this notebook helpful or have any suggestions for improvement, please consider upvoting, forking the notebook, and making changes. Your feedback and contributions are greatly appreciated!

## Follow and Visit My Other Works 📚🔍
### Don't forget to [follow me on Kaggle](https://www.kaggle.com/saswattulo) to stay updated with my latest works. You can also visit my other notebooks and kernels for more insights and tutorials.

## Perform Hyperparameter Tuning 🛠️

### To further optimize the performance of the model, you can experiment with hyperparameter tuning. This involves adjusting parameters such as learning rate, batch size, and optimizer settings to find the best configuration for your specific task and dataset.
### Feel free to explore different hyperparameter values and techniques such as grid search or random search to find the optimal combination. You can refer to my notebook on hyperparameter tuning for guidance and inspiration.

[Link to my notebook on Hyperparameter Tuning](https://www.kaggle.com/code/saswattulo/keras-tuner-explained-in-detail)

### Happy tuning! 🚀
